Write a program that goes to a photo-sharing site like Flickr or Imgur, searches for a category of photos, and then downloads all the resulting images. You could write a program that works with any photo site that has a search feature.

In [1]:
#! /usr/bin/env python3
import requests, os, bs4
from urllib.request import quote
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import logging
import time

In [2]:
os.chdir('/Users/paternhong/Desktop')

In [3]:
choose = input("Where do you want to find pics?? Flickr: enter 1, Imgur: enter 2.\n")
search = input("Search what kind of pic?\n")
# 創建放置圖片的資料夾
os.makedirs(search, exist_ok=True)
# 紀錄所有取得的圖片連結
file_urls = open(os.path.join(search, 'all_urls.txt'), 'w')
# 簡單記錄狀況
file_log = open(os.path.join(search, 'log.txt'), 'w')
'''
logging.basicConfig(filename= os.path.join(search, 'detail_log.txt'), level=logging.DEBUG,
                    format=' %(asctime)s - %(levelname)s - %(message)s')
有需要detail_log再開啟(包含)
'''
print('========Mission Start========\n{}'.format(time.strftime("%Y-%m-%d %H:%M:%S")))
file_log.write('========Mission Start========\n{}\n'.format(time.strftime("%Y-%m-%d %H:%M:%S")))

def downLoad(search_name, _urls, _log):
    print("Total {} files need to download.".format(len(_urls)))
    for i in range(len(_urls)):
        try:
            img_name = search_name + '_{}.jpg'.format(i + 1) # 像編號從一開始
            print('Downloading {} from "{}"...'.format(img_name, _urls[i]))
            _log.write('{} - Downloading {} from "{}"...\n'.format(time.strftime("%Y-%m-%d %H:%M:%S"), img_name, _urls[i]))


            img = requests.get(_urls[i])
            img.raise_for_status()
            imageFile = open(os.path.join(search_name, img_name), 'wb')
            for chunk in img.iter_content(100000):
                imageFile.write(chunk)
            time.sleep(2)
            imageFile.close()
            print('Download {} from "{}": PASS'.format(img_name, _urls[i]))
            _log.write('{} - Download {} from "{}": PASS\n'.format(time.strftime("%Y-%m-%d %H:%M:%S"), img_name, _urls[i]))

        # 發現無法get img時候執行
        except requests.exceptions.MissingSchema:
            print('Url {} is missing.'.format(_urls[i]))
            _log.write('{} - Url "{}" is missing.\n'.format(time.strftime("%Y-%m-%d %H:%M:%S"), _urls[i]))

        except requests.exceptions.InvalidURL:
            print('Url {} is Invalid.'.format(_urls[i]))
            _log.write('{} - Url {} is Invalid.\n'.format(time.strftime("%Y-%m-%d %H:%M:%S"), _urls[i]))

        except requests.exceptions.HTTPError:
            print('503 Server Error: Backend unavailable with Url {}.'.format(_urls[i]))
            _log.write('{} - 503 Server Error: Backend unavailable with Url {}.\n'.format(time.strftime("%Y-%m-%d %H:%M:%S"), _urls[i]))
        except:
            print('Download {} from "{}": FAIL'.format(img_name, _urls[i]))
            _log.write('{} - Download {} from "{}": FAIL\n'.format(time.strftime("%Y-%m-%d %H:%M:%S"), img_name, _urls[i]))

pic_link_list = []
if choose == '1':
    browser = webdriver.Firefox()
    browser.implicitly_wait(10)
    browser.get('https://www.flickr.com/search/?text=' + quote(search) + '&view_all=1')
    
    htmlElem = browser.find_element_by_tag_name('html')
    # 搜尋結果是動態的，所以要先擴展到後面
    for i in range(10):
        htmlElem.send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
    picAll = browser.find_elements_by_xpath('//*[@id="search-unified-content"]/div/div/div[2]/div/div/div/a')
    # logging.info('There are {} results at this trial.'.format(len(picAll)))
    print('There are {} results at this trial.'.format(len(picAll)))
    file_log.write('{} - There are {} results at this trial.\n'.format(
        time.strftime("%Y-%m-%d %H:%M:%S"), len(picAll)))
    
    # 進入每個結果取得下載連結並存進 pic_link_list
    for i in range(len(picAll)):
        # 用新分頁開啟各連結
        link = picAll[i].get_attribute('href')
        new_tab = 'window.open("{}");'.format(link)
        browser.execute_script(new_tab)
        # 切換到此新頁面
        browser.switch_to_window(browser.window_handles[-1])
        time.sleep(3)
        # 在新分頁尋找下載連結
        try:
            down_bu = WebDriverWait(browser, 10, 0.5).until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="content"]/div/div[2]/div/div[5]/div[3]')))
            down_bu.click()
            time.sleep(2)
            # 下載原始圖檔
            down_url = browser.find_elements_by_xpath('//*[@class="原本大小"]/a')
            if down_url == []:
                # logging.warning('Unable to get download url from "{}"'.format(link))
                print('Unable to get download url from "{}"'.format(link))
                file_log.write('{} - Unable to get download url from "{}"\n'.format(
                    time.strftime("%Y-%m-%d %H:%M:%S"), link))
                # 關閉此新頁面
                browser.close()
                # 返回原本頁面
                browser.switch_to_window(browser.window_handles[0])
                time.sleep(1)
                continue
            down_url = down_url[0].get_attribute('href')
            # logging.info('Get download url {} from "{}"'.format(down_url, link))
            print('Get download url {} from "{}"'.format(down_url, link))
            file_log.write('{} - Get download url {} from "{}"\n'.format(
                time.strftime("%Y-%m-%d %H:%M:%S"), down_url, link))
            # 把抓取到的圖片連結存到list裡
            pic_link_list.append(down_url)
            file_urls.write('From: {}\nGet Url: {}\n'.format(link, down_url))
            
        except:
            # logging.warning('FAIL to get download url from "{}"'.format(link))
            print('FAIL to get download url from "{}"'.format(link))
            file_log.write('{} - FAIL to get download url from "{}"\n'.format(
                time.strftime("%Y-%m-%d %H:%M:%S"), link))
        # 關閉此新頁面
        browser.close()
        # 返回原本頁面
        browser.switch_to_window(browser.window_handles[0])
        time.sleep(1)
        

# 下載圖片
downLoad(search, pic_link_list, file_log)
    

print('========Mission Complete========\n{}'.format(time.strftime("%Y-%m-%d %H:%M:%S")))
file_log.write('========Mission Complete========\n{}\n'.format(time.strftime("%Y-%m-%d %H:%M:%S")))
file_urls.close()
file_log.close()
browser.quit()

Where do you want to find pics?? Flickr: enter 1, Imgur: enter 2.
1
Search what kind of pic?
大阪
========Mission Start========
2019-09-26 13:32:56
There are 200 results at this trial.


/Users/paternhong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: DeprecationWarning: use driver.switch_to.window instead


Get download url https://live.staticflickr.com/3688/8856359202_fcb310a0d9_o_d.jpg from "https://www.flickr.com/photos/69582595@N08/8856359202/"


/Users/paternhong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:114: DeprecationWarning: use driver.switch_to.window instead


FAIL to get download url from "https://www.flickr.com/photos/aquaring/4060073016/"
Get download url https://live.staticflickr.com/4017/4536057969_0cccd3b318_o_d.jpg from "https://www.flickr.com/photos/fmhsu/4536057969/"
Unable to get download url from "https://www.flickr.com/photos/130075302@N08/34935939244/"


...

Unable to get download url from "https://www.flickr.com/photos/hide-i/4502010792/"
Unable to get download url from "https://www.flickr.com/photos/nakaneko/3512545209/"
Unable to get download url from "https://www.flickr.com/photos/124591456@N03/46035015534/"
Unable to get download url from "https://www.flickr.com/photos/kuriboz/4386743791/"
Get download url https://live.staticflickr.com/7174/6572123533_e903a153c0_o_d.jpg from "https://www.flickr.com/photos/71464207@N02/6572123533/"
Get download url https://live.staticflickr.com/8229/8543973211_1d6913e051_o_d.jpg from "https://www.flickr.com/photos/u3kjp/8543973211/"
Get download url https://live.staticflickr.com/8595/15945100208_f16396e6a8_o_d.jpg from "https://www.flickr.com/photos/129607005@N05/15945100208/"
Unable to get download url from "https://www.flickr.com/photos/hide-i/4501370261/"
Get download url https://live.staticflickr.com/7141/6520895009_c4259781b5_o_d.jpg from "https://www.flickr.com/photos/wesllii/6520895009/"
Unable 

Unable to get download url from "https://www.flickr.com/photos/sbisaro/16669224462/"
Get download url https://live.staticflickr.com/7901/45955525925_9d03de6615_o_d.jpg from "https://www.flickr.com/photos/144232687@N05/45955525925/"
Get download url https://live.staticflickr.com/65535/46699570335_e62fc4a253_o_d.jpg from "https://www.flickr.com/photos/169490374@N07/46699570335/"
Get download url https://live.staticflickr.com/7880/31865770667_85d5bede8c_o_d.jpg from "https://www.flickr.com/photos/161140480@N02/31865770667/"
Unable to get download url from "https://www.flickr.com/photos/natsuki_harada/35356766854/"
Get download url https://live.staticflickr.com/4890/31865773037_9df9685fa4_o_d.jpg from "https://www.flickr.com/photos/161140480@N02/31865773037/"
Get download url https://live.staticflickr.com/6033/6248901824_13963f37c5_o_d.jpg from "https://www.flickr.com/photos/inucara/6248901824/"
Get download url https://live.staticflickr.com/5256/5483157841_dd56c4da65_o_d.jpg from "https:/

Download 大阪_5.jpg from "https://live.staticflickr.com/7070/27279692663_dba1146bf6_o_d.jpg": PASS
Download 大阪_6.jpg from "https://live.staticflickr.com/7533/27279692253_838e1b4929_o_d.jpg": PASS
Download 大阪_7.jpg from "https://live.staticflickr.com/7300/27815242891_1161463d56_o_d.jpg": PASS
Download 大阪_8.jpg from "https://live.staticflickr.com/4158/34395199751_cbe64a91ff_o_d.jpg": PASS
Download 大阪_9.jpg from "https://live.staticflickr.com/2644/4011071543_240f685604_o_d.jpg": PASS
Download 大阪_10.jpg from "https://live.staticflickr.com/4072/4536068797_4b5f743a34_o_d.jpg": PASS
Download 大阪_11.jpg from "https://live.staticflickr.com/8186/8395406816_bfe2282f89_o_d.jpg": PASS
Download 大阪_12.jpg from "https://live.staticflickr.com/2801/4410006511_31b3371206_o_d.jpg": PASS
Download 大阪_13.jpg from "https://live.staticflickr.com/4063/4387467194_2ef496bacf_o_d.jpg": PASS
Download 大阪_14.jpg from "https://live.staticflickr.com/2633/4011069919_f6111feabf_o_d.jpg": PASS
Download 大阪_15.jpg from "https:

Download 大阪_48.jpg from "https://live.staticflickr.com/7290/8744728372_7b1dce1a1a_o_d.jpg": PASS
Download 大阪_49.jpg from "https://live.staticflickr.com/7174/6572123533_e903a153c0_o_d.jpg": PASS
Download 大阪_50.jpg from "https://live.staticflickr.com/8229/8543973211_1d6913e051_o_d.jpg": PASS
Download 大阪_51.jpg from "https://live.staticflickr.com/8595/15945100208_f16396e6a8_o_d.jpg": PASS
Download 大阪_52.jpg from "https://live.staticflickr.com/7141/6520895009_c4259781b5_o_d.jpg": PASS
Download 大阪_53.jpg from "https://live.staticflickr.com/4166/34525371775_b2fc038591_o_d.jpg": PASS
Download 大阪_54.jpg from "https://live.staticflickr.com/8645/30384923562_b78b1608e8_o_d.jpg": PASS
Download 大阪_55.jpg from "https://live.staticflickr.com/7068/14117048522_c257f36382_o_d.jpg": PASS
Download 大阪_56.jpg from "https://live.staticflickr.com/4192/34077624150_89bcf59e96_o_d.jpg": PASS
Download 大阪_57.jpg from "https://live.staticflickr.com/7230/7186891204_1670f631a8_o_d.jpg": PASS
Download 大阪_58.jpg from "

Download 大阪_90.jpg from "https://live.staticflickr.com/7880/31865770667_85d5bede8c_o_d.jpg": PASS
Download 大阪_91.jpg from "https://live.staticflickr.com/4890/31865773037_9df9685fa4_o_d.jpg": PASS
Download 大阪_92.jpg from "https://live.staticflickr.com/6033/6248901824_13963f37c5_o_d.jpg": PASS
Download 大阪_93.jpg from "https://live.staticflickr.com/5256/5483157841_dd56c4da65_o_d.jpg": PASS
Download 大阪_94.jpg from "https://live.staticflickr.com/7226/7186877108_21ba21f9fb_o_d.jpg": PASS
Download 大阪_95.jpg from "https://live.staticflickr.com/7081/7186868636_13eb314186_o_d.jpg": PASS
Download 大阪_96.jpg from "https://live.staticflickr.com/2887/33464984790_9b08eb7500_o_d.jpg": PASS
Download 大阪_97.jpg from "https://live.staticflickr.com/4033/4435632684_750b3c6db4_o_d.jpg": PASS
Download 大阪_98.jpg from "https://live.staticflickr.com/229/494515463_83554802eb_o_d.jpg": PASS
Download 大阪_99.jpg from "https://live.staticflickr.com/7179/6996069760_5b2eeb0868_o_d.jpg": PASS
Download 大阪_100.jpg from "htt